In [ ]:
import numpy as np
import scipy as sp
import scipy.signal as sig
import altair as alt
import pandas as pd
from pathlib import Path  # pathlib is seriously awesome!
import sys
import sklearn.decomposition as sld

In [ ]:
DRS_file=pd.read_csv('All_RSC_Data.csv')
rs_file=pd.ExcelFile('2023-07-14_random_sample.xlsx')
#rs stands for random sample
rs_sheet_info=pd.read_excel(rs_file,'samples-354-U1449_U1450_U1451_U')
rs_sheet_lith=pd.read_excel(rs_file,'random_sample')
#rs_sheet = the specific sheet using to merge
MiddleDepth=(rs_sheet_info['Top depth CSF-A (m)']+rs_sheet_info['Bottom depth CSF-A (m)'])/2
rs_sheet_info.insert(11,'depth-m',MiddleDepth)
#calculating middle depth from top and bottom depth of core section and inserting values into sheet as new column
DRS_data=pd.merge(rs_sheet_info,DRS_file,on=['Site','Hole','depth-m'])
DRS_data=pd.merge(DRS_data,rs_sheet_lith,left_on=['ID'],right_on=['name'],how='left')
DRS_data.to_csv('DRS_data.csv',index=False)